In [1]:
import random
import numpy as np
import time
import math
import uuid
import csv
import os
from math import cos, exp, sqrt, pi

In [2]:
POP_SIZE = 100
N_EVALS = 100000
N_REPLICATES = 30
MIN_LEN = 3
MAX_LEN = 100
gene_low, gene_high = -2.0, 2.0

time_str = time.strftime('%Y%m%d_%H%M%S')
out_dir = f'GA_Variable_results_{time_str}'
os.makedirs(out_dir, exist_ok=True)
csv_path = os.path.join(out_dir, 'GA_Variable_results.csv')

In [3]:
def mkPenalizer(d_target, lam=10, deadzone=2):
    def _pen(ind):
        dev = abs(len(ind) - d_target)
        return 0 if dev <= deadzone else lam * dev
    return _pen

def loadSantaFeMap(path='santafe_trail.txt'):
    world_rows = []
    start_x = start_y = None
    with open(path) as f:
        for y, line in enumerate(f):
            row = []
            for x, c in enumerate(line.rstrip("\n")):
                if c == '#':
                    row.append(1)
                elif c in ('.','S'):
                    row.append(0)
                    if c == 'S':
                        start_x, start_y = x, y
                else:
                    continue
            if row:
                world_rows.append(row)
    if not world_rows:
        raise ValueError(f'No valid rows in Santa Fe map: {path}')
    H = len(world_rows)
    W = len(world_rows[0])
    for r in world_rows:
        if len(r) != W:
            raise ValueError('Inconsistent row lengths in Santa Fe map')
    world = [c for row in world_rows for c in row]
    if start_x is None or start_y is None:
        start_x = start_y = 0
    return world, W, H, start_x, start_y

world, W, H, START_X, START_Y = loadSantaFeMap()

In [4]:
def generateInd():
    length = random.randint(MIN_LEN, MAX_LEN)
    return [random.uniform(gene_low, gene_high) for _ in range(length)]

def initPop():
    return [generateInd() for _ in range(POP_SIZE)]

In [5]:
def sphere(d_target, lam):
    pen = mkPenalizer(d_target, lam)
    return lambda ind: sum(x**2 for x in ind) + pen(ind)

def rastrigin(d_target, lam):
    pen = mkPenalizer(d_target, lam)
    return lambda ind: 10*len(ind) + sum(x**2 - 10*cos(2*pi*x) for x in ind) + pen(ind)

def griewank(d_target, lam):
    pen = mkPenalizer(d_target, lam)
    def f(ind):
        sum_sq = sum(x**2 for x in ind)
        prod_cos = np.prod([cos(x/sqrt(i+1)) for i,x in enumerate(ind)])
        return 1 + sum_sq/4000.0 - prod_cos + pen(ind)
    return f

def ackley(d_target, lam):
    pen = mkPenalizer(d_target, lam)
    def f(ind):
        d = len(ind)
        sum_sq = sum(x**2 for x in ind)
        sum_cos = sum(cos(2*pi*x) for x in ind)
        term1 = -20 * exp(-0.2 * sqrt(sum_sq/d))
        term2 = -exp(sum_cos/d)
        return term1 + term2 + 20 + math.e + pen(ind)
    return f

def koza(d_target, lam):
    pen = mkPenalizer(d_target, lam)
    x_vals = np.linspace(-1,1,20)
    def f(ind):
        err = 0.0
        for x in x_vals:
            y_t = x**4 + x**3 + x**2 + x
            y_p = sum(w*(x**i) for i,w in enumerate(ind))
            err += (y_p - y_t)**2
        return err + pen(ind)
    return f

def knapsack(d_target, lam, items, capacity):
    pen = mkPenalizer(d_target, lam)
    def f(ind):
        sel = [int(round(x))%2 for x in ind[:len(items)]]
        w_sum = sum(w for (w,v),s in zip(items, sel) if s)
        v_sum = sum(v for (w,v),s in zip(items, sel) if s)
        over = max(0, w_sum - capacity)
        return -v_sum + 1000*over + pen(ind)
    return f

def parity(N, lam):
    d_target = 2**N
    pen = mkPenalizer(d_target, lam)
    def f(ind):
        table = [int(round(x))%2 for x in ind[:d_target]]
        errs = 0
        for i in range(d_target):
            target = bin(i).count('1') % 2
            if i < len(table):
                if table[i] != target:
                    errs += 1
            else:
                errs += 1
        return errs + pen(ind)
    return f

def santaFe(d_target, lam):
    pen = mkPenalizer(d_target, lam)
    def f(ind, max_steps=400):
        direction = 0
        x,y = START_X, START_Y
        food = 0
        visited = world.copy()
        idx = lambda xx,yy: (yy%H)*W + (xx%W)
        for i in range(min(len(ind), max_steps)):
            if visited[idx(x,y)] == 1:
                food += 1
                visited[idx(x,y)] = 0
            action = int(abs(ind[i])) % 3
            if action == 0:
                dx,dy = [(0,1),(1,0),(0,-1),(-1,0)][direction]
                x += dx; y += dy
            elif action == 1:
                direction = (direction + 1) % 4
            else:
                direction = (direction - 1) % 4
        return -food + pen(ind)
    return f

In [6]:
def tournamentSelection(pop, fit, k=3):
    sel = random.sample(pop, k)
    return min(sel, key=fit)

def rankSelection(pop, fit):
    spop = sorted(pop, key=fit)
    probs = np.linspace(1,0.1,POP_SIZE)
    probs /= probs.sum()
    return spop[np.random.choice(POP_SIZE, p=probs)]

def crossover(p1, p2, p_cross):
    if random.random() > p_cross:
        return p1[:], p2[:]
    cut1 = random.randint(1, len(p1)-1)
    cut2 = random.randint(1, len(p2)-1)
    c1 = p1[:cut1] + p2[cut2:]
    c2 = p2[:cut2] + p1[cut1:]
    for c in (c1,c2):
        while len(c) < MIN_LEN:
            c.append(random.uniform(gene_low, gene_high))
    return c1[:MAX_LEN], c2[:MAX_LEN]

def mutate(ind, p_mut):
    ind = ind[:]
    if random.random() < 0.3:
        if random.random() < 0.5 and len(ind) < MAX_LEN:
            ind.append(random.uniform(gene_low, gene_high))
        elif len(ind) > MIN_LEN:
            del ind[random.randrange(len(ind))]
    for i in range(len(ind)):
        if random.random() < p_mut:
            ind[i] += random.gauss(0,0.5)
            ind[i] = max(min(ind[i], gene_high), gene_low)
    return ind

In [7]:
def runGA(fit_func, name, selection='tournament'):
    pop = initPop()
    evals = 0
    best_hist = []
    while evals < N_EVALS:
        new_pop = []
        gen_idx = len(best_hist)
        p_mut = 0.05 * (1 - gen_idx/(N_EVALS/POP_SIZE))
        p_cross = 0.8 * (1 - 0.5*gen_idx/(N_EVALS/POP_SIZE))
        elite = min(pop, key=fit_func)
        new_pop.append(elite)
        while len(new_pop) < POP_SIZE and evals < N_EVALS:
            if selection=='tournament':
                p1 = tournamentSelection(pop, fit_func)
                p2 = tournamentSelection(pop, fit_func)
            else:
                p1 = rankSelection(pop, fit_func)
                p2 = rankSelection(pop, fit_func)
            c1, c2 = crossover(p1,p2,p_cross)
            new_pop.append(mutate(c1,p_mut))
            if len(new_pop)<POP_SIZE:
                new_pop.append(mutate(c2,p_mut))
        pop = sorted(new_pop, key=fit_func)[:POP_SIZE]
        evals += POP_SIZE
        best = fit_func(elite)
        best_hist.append(best)
        if gen_idx % 10 == 0:
            print(f'{name} Evals {evals}: Best = {best:.6f}, Length = {len(elite)}')
    return elite, best_hist

In [8]:
with open(csv_path, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=[
        'run_id','benchmark','replicate','seed','best','len_final','cpu_s'
    ])
    writer.writeheader()

    tmp_items = [(random.randint(1,20), random.randint(10,100)) for _ in range(20)]
    benchmarks = {
        'Sphere': sphere(30,10),
        'Rastrigin': rastrigin(30,10),
        'Griewank': griewank(30,10),
        'Ackley': ackley(30,10),
        'Koza': koza(10,10),
        'Knapsack': knapsack(20,0,tmp_items,50),
        'Parity': parity(6,10),
        'SantaFe': santaFe(100,10),
    }

    for name, func in benchmarks.items():
        for rep in range(N_REPLICATES):
            seed = random.randint(0,2**31-1)
            random.seed(seed); np.random.seed(seed)
            start = time.perf_counter()
            best_ind, hist = runGA(func, name, selection='rank')
            cpu = time.perf_counter() - start
            writer.writerow({
                'run_id': uuid.uuid4(),
                'benchmark': name,
                'replicate': rep,
                'seed': seed,
                'best': round(func(best_ind),6),
                'len_final': len(best_ind),
                'cpu_s': round(cpu,4)
            })
print(f'Results written to {csv_path}')

Ackley Evals 100: Best = 4.709969, Length = 30
Ackley Evals 1100: Best = 4.708416, Length = 30
Ackley Evals 2100: Best = 4.514495, Length = 31
Ackley Evals 3100: Best = 4.027076, Length = 30
Ackley Evals 4100: Best = 3.898208, Length = 30
Ackley Evals 5100: Best = 3.784830, Length = 29
Ackley Evals 6100: Best = 3.639314, Length = 31
Ackley Evals 7100: Best = 3.639314, Length = 31
Ackley Evals 8100: Best = 3.429185, Length = 29
Ackley Evals 9100: Best = 3.417615, Length = 31
Ackley Evals 10100: Best = 3.164505, Length = 30
Ackley Evals 11100: Best = 3.164505, Length = 30
Ackley Evals 12100: Best = 3.164505, Length = 30
Ackley Evals 13100: Best = 3.076198, Length = 28
Ackley Evals 14100: Best = 3.026995, Length = 28
Ackley Evals 15100: Best = 2.924823, Length = 29
Ackley Evals 16100: Best = 2.814995, Length = 29
Ackley Evals 17100: Best = 2.709962, Length = 31
Ackley Evals 18100: Best = 2.679658, Length = 29
Ackley Evals 19100: Best = 2.237040, Length = 32
Ackley Evals 20100: Best = 2.14